In [ ]:
import numpy as np
import mdtraj as md
from scipy.spatial.transform import Rotation

from simtk import unit
from simtk.openmm import Vec3, LangevinIntegrator, MonteCarloAnisotropicBarostat
from simtk.openmm.app import HBonds, NoCutoff, GBn2
from simtk.openmm.app import ForceField, Modeller, Simulation
from simtk.openmm.app import PDBReporter, DCDReporter, StateDataReporter

from parmed.openmm.reporters import RestartReporter
from openforcefield.topology import Molecule, Topology
from openmoltools.utils import smiles_to_mdtraj_ffxml

In [ ]:
FFXML_DIR = './'

STEP_SIZE = 250
TOTAL_STEPS = 1000
SIM_DURATION = TOTAL_STEPS*STEP_SIZE

ENERGY_TOLERANCE = 0.02
MAX_ENERGY_ITERS = 5000

LANGE_TOLERANCE = 10**-5
LANGE_TEMPERATURE = 302.0*unit.kelvin
LANGE_FRICTION = 1.0/unit.picoseconds
LANGE_STEPSIZE = 2.0*unit.femtoseconds

RENDER_INTERVAL = 5

PDB_FRAME_DIR= 'pdb-frames'
PDB_REPORT_FILE = 'simulation-GBn2.pdb'
DCD_REPORT_FILE = 'simulation-GBn2.dcd'
STATE_REPORT_FILE = 'simulation-GBn2.state.txt'

STEP_PER_REPORT = 100
RESTART_FILE = 'restart-GBn2.rst7'
STATE_RESTART_FILE = 'restart-GBn2.xml'
CHECKPOINT_RESTART_FILE = 'restart-GBn2.chk'

In [ ]:
reporters = [
    PDBReporter(PDB_REPORT_FILE, STEP_PER_REPORT),
    DCDReporter(DCD_REPORT_FILE, STEP_PER_REPORT),
    StateDataReporter(
        STATE_REPORT_FILE, STEP_PER_REPORT, 
        progress=True, temperature=True, 
        potentialEnergy=True, density=True,
        totalSteps=TOTAL_STEPS, speed=True)]

In [ ]:
cdtTRAJ = md.load('CDT-0.gaff.mol2')
cdtTRAJ_top = cdtTRAJ[0].top.to_openmm()
cdtTRAJ_xyz = cdtTRAJ[0].openmm_positions(0)

ligTRAJ = md.load('LIG-0.gaff.mol2')
ligTRAJ_top = ligTRAJ[0].top.to_openmm()
ligTRAJ_xyz = ligTRAJ[0].openmm_positions(0)

model = Modeller(cdtTRAJ_top, cdtTRAJ_xyz)
model.add(ligTRAJ_top, ligTRAJ_xyz)

In [ ]:
print('Loading forcefields...')
forcefield = ForceField(
    FFXML_DIR + 'LIG-0.xml', 
    FFXML_DIR + 'CDT-0.xml')

print('Creating system...')
system = forcefield.createSystem(
    model.topology,
    nonbondedMethod=NoCutoff,
    polarization='mutual',
    mutualInducedTargetEpsilon=0.0005,
    implicitSolvent=GBn2,
    implicitSolventSaltConc=0.5*unit.moles/unit.liter)

In [ ]:
integrator = LangevinIntegrator(LANGE_TEMPERATURE, LANGE_FRICTION, LANGE_STEPSIZE)
integrator.setConstraintTolerance(LANGE_TOLERANCE)

print('Building simulation environment...\n')

simulation = Simulation(model.topology, system, integrator) 
simulation.context.setPositions(model.positions)

print('Initial system energy:\n\t%s\n' % echo_context_energy(simulation.context))

simulation.minimizeEnergy(ENERGY_TOLERANCE, MAX_ENERGY_ITERS)

print('System energy after minimization:\n\t%s\n' % echo_context_energy(simulation.context))

for r in reporters: simulation.reporters.append(r)

In [ ]:
for (i,s) in enumerate(range(STEP_SIZE, SIM_DURATION, STEP_SIZE)): 
    simulation.step(STEP_SIZE)
    print('System energy at step %s (%sfs):\n\t%s' % (i, s, echo_context_energy(simulation.context)))